In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

import sys
sys.path.append(r"C:\Users\rapha\My Drive\Work\data_science\projects\scratch_nn")
import nn

In [3]:
with open(r"C:\Users\rapha\Documents\datasets\tiny_stories\TinyStoriesV2-GPT4-train.txt", 'r', encoding='utf-8') as f:
  train = f.read()

In [2]:
with open(r"C:\Users\rapha\Documents\datasets\tiny_stories\TinyStoriesV2-GPT4-valid.txt", 'r', encoding='utf-8') as f:
  valid = f.read()

In [3]:
def corpus_to_documents(str):
  str = str.replace("\\","")
  docs = str.split("<|endoftext|>")
  return docs

In [4]:
# train_docs = corpus_to_documents(train)
val_docs = corpus_to_documents(valid)

In [5]:
test_docs = val_docs[:1000]

In [6]:
tokenizer = nn.text.BytePairTokenizer(1000)
vectorizer = nn.text.Vectorizer()

In [7]:
tokenizer.fit(test_docs)

In [8]:
val_tokens = tokenizer.transform2(test_docs)

KeyboardInterrupt: 

In [13]:
vectorizer.fit(val_tokens)

In [14]:
val_sequences = vectorizer.transform(val_tokens)

In [25]:
val_sequences[0][:10]

[97, 401, 124, 133, 6, 49, 232, 44, 3, 578]

In [8]:
tokenizer = nn.text.BytePairTokenizer(1000)
vectorizer = nn.text.Vectorizer()

print(f"Fitting tokenizer...")
tokenizer.fit(train)
print(f"Tokenizer fit compelte.")

train_tokens = tokenizer.transform(clean_train_docs)
val_tokens = tokenizer.transform(clean_val_docs)

print(f"Fitting vectorizer...")
vectorizer.fit(train_tokens)
print(f"Vectorizer fit compelte.")
train_sequences = vectorizer.transform(train_tokens)
val_sequences = vectorizer.transform(val_tokens)

In [17]:
def targets_from_sequence(sequence, context_size):
  """
  Constructs n times X, y training data from a sequence. X being our
  context_size input,  y the target, and n the amount of targets created from a
  sequence (should be equal to len(sequence) - 1, since every word can be a
  target except the first)

  X is context_size words used to predict context_size + 1 word y

  Args:
    sequence (array): array of ints, vectorised tokens
    context_size (int): length of X for each X, y pair returned
  
  Returns:
    list[(X,y)]: n long list containing tuples of X, y pairs
  """
  target_amount = len(sequence) - 1

  # Ensure input in an np.ndarray
  sequence = np.array(sequence)

  # Result array is a matrix for (n, (X,y))
  # n = target_amount, X = context_size, y = 1
  result = np.zeros((target_amount, context_size+1))


  for i in range(target_amount):
    # Get index of context start, unless its negative
    idx = i - context_size + 1
    context_start = max(0, idx)

    # Get sequence and target, ensuring they are arrays
    context = sequence[context_start:i+1]
    target = np.array(sequence[i+1], ndmin=1)

    # If our context is shorter than context size, we will pad context
    if context_start == 0:
      context = np.pad(context, (0, np.abs(idx)), mode='constant')

    result[i,:] = np.concatenate([context, target])
    
  return result

def targets_from_sequences(sequences, context_size):
  """Calls targets_from_sequences() on a list of sequences"""
  result = None
  for sequence in sequences:
    targets = targets_from_sequence(sequence, context_size)
    result = np.concatenate([result, targets]) if result is not None else targets
  return result


In [ ]:
train_data = targets_from_sequences(train_sequences)
val_data = targets_from_sequences(val_sequences)

In [19]:
val_data = targets_from_sequences(val_sequences, 64)

In [20]:
val_data

array([[ 97.,   0.,   0., ...,   0.,   0., 401.],
       [ 97., 401.,   0., ...,   0.,   0., 124.],
       [ 97., 401., 124., ...,   0.,   0., 133.],
       ...,
       [ 38.,   1.,  20., ..., 811., 288., 704.],
       [  1.,  20., 695., ..., 288., 704.,   1.],
       [ 20., 695.,  19., ..., 704.,   1.,   7.]])